# Gray-Box Modeling of Guitar Amplifiers

## 0. Introduction

- 歪み回路のモデリングは大きく分けて３つの手法がある
  - White box modeling
  - Black(Gray) box modeling

- White box modeling
    - 対象システムの詳細な情報を元にモデリングする
    - 回路などの微分方程式を解いたり、状態空間モデルを組んだり
    - 精度は高いが、回路を熟知していないと難しい
        - 特に非線形素子は解析が難しい
    - 計算コストも高い
 
- WeinerモデルとHammersteinモデル
    - Weinerモデル
        - Linear-NonLinear
    - Hammersteinモデル
        - NonLinear-Linear
    - Weiner-Hammersteinモデル
        - Linear-NonLinear-Linear

- 線形ブロックはメモリを持つ、非線形モデルはメモリを持たないといわれることがある

- Black(Gray) box modeling
    - Block based model
        - WeinerモデルとHammersteinモデルを組み合わせる
    - [A. Novak et al. ](https://www.researchgate.net/profile/Antonin_Novak/publication/228818778_Chebyshev_model_and_Synchronized_Swept_Sine_Method_in_nonlinear_audio_effect_modeling/links/0fcfd5086a31d4cb83000000.pdf)
    は各ブランチが入力の各周波数成分を表すようなマルチブランチのHammersteinモデルを提案
        - 入力信号の振幅がシステム同定の際に用いた信号の振幅と一致しないと上手くシミュレーション出来ない
    - Fractal AudioやKemperもBlock based modelを使っていたりする
    - 著者はいくつかブロックモデルを提案していたが、本論文では１回のみのパラメータ最適化のステップでモデリングできるものを提案している。

## 1. Digital Model

- ギターアンプの基本は歪み要素をフィルターで挟むこと
    - Wiener-Hammersteinモデルでこれを再現
- 前著では２つの非線形ブロックを用いた
    - 完璧を追い求めるよりも計算コストを下げるほうが重要


- 入力フィルタ$H_1(z)$
    - 周波数ごとの歪みの量を決める
    - ある周波数で減衰が大きいと、あとに来る歪みブロック内のマッピング関数で飽和しない
    - 相互変調歪みの量も決める
    
    
- 非線形ブロック
    - $g_{pre}x(n) + g_{bias}$はカソードの電圧変化による陽極電流による入力依存のバイアスシフトをモデリング
    - 歪み成分のシミュレーションには$\tanh$をベースとしたマッピング関数を用い、4つのパラメータがある
    - この論文内での非線形ブロックは通常にはないローパスフィルターを含む(extended Winer-Hammersteinモデルの由来)
    
- 出力フィルタ$H_2(z)$
    - 歪み信号に独自の音色をもたせるため、倍音のスペクトル形状を決定づける
 
 $p = (p_{H, 1}, p_{nl}, p_{H, 2})^T$を最適化する

## 2. Measurement Setup

- デジタル振幅±1が±1Vに対応するように構成
- output1 -> アンプ -> dummy load -> input1
- output2 -> input2
    - 補正フィルタ作成のため

## 3. System Identification

- extended Wiener-Hammersteinモデルの最適化は線形ブロックの最適化と非線形ブロックの最適化に分けられる
    - 両方Levenberg-Marquardt法で最適化する

- Swept Sine信号で$H_1(z), H_2(z)$を測定
    - 0.01Vより低い電圧で測定、線形部分のインパルス応答を$h_{low}(n)$とする
    - 1Vより低い電圧で測定、線形部分をインパルス応答$h_{high}(n)$とする

- 非線形ブロックは$H_1$の寄与を無効にする働きがある。
    - すべての周波数でそうなるわけではない

- 得られたインパルス応答を離散フーリエ変換して$H_1, H_2$を算出
    - $H_1(z) = \frac{H_{low}(z)}{H_{high}(z)}$
    - $H_2(z) = H_{high}(z)$
    
### 3.1 Adapting Second-Order IIR Filters
- 適切な周波数の解像度を得るために、長いインパルス応答を測る必要がある
- FIRよりIIRの方がタップ数を減らせるので計算量を減らせる
- 最適な2次のIIRフィルタの集合を得るため、反復的に(iteratively)パラメータを調整する
- HFS, LFS, ピークフィルタを直列に繋がれる
- 